In [1]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout,PReLU,BatchNormalization
from tensorflow.keras.preprocessing.image import img_to_array, load_img,ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping
from keras.api._v2.keras.callbacks import TensorBoard

2024-04-07 22:43:35.982017: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 22:43:36.022261: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 22:43:36.022299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 22:43:36.023232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 22:43:36.029864: I tensorflow/core/platform/cpu_feature_guar

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
training_set = train_datagen.flow_from_directory('Images', 
                                                 target_size = (256, 256), 
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 #color_mode = 'grayscale',
                                                 shuffle = True,
                                                 subset = "training")

test_set = train_datagen.flow_from_directory('Images',
                                                   target_size = (256, 256),
                                                   batch_size = 16,
                                                   class_mode = 'categorical',
                                                   #color_mode = 'grayscale',
                                                   shuffle = False,
                                                   subset = "validation")

Found 7200 images belonging to 9 classes.
Found 1800 images belonging to 9 classes.


In [3]:
input_shape= (256,256,3)
model_3 = Sequential([
    
    Conv2D(16,(3,3), activation = 'leaky_relu', padding ='same', input_shape = input_shape),
    
    MaxPooling2D((2,2)),
    
    Conv2D(32,(3,3), activation = 'leaky_relu', padding ='same'),
    #Conv2D(16,(3,3), activation = 'leaky_relu', padding ='same'),

    MaxPooling2D((2,2)),
    
    
    Conv2D(32,(3,3), activation = 'leaky_relu', padding='same'),

    MaxPooling2D((2,2)),
    # Conv2D(52,(3,3),activation = 'leaky_relu', padding = 'same', strides =(2,2)),
    
    Conv2D(64,(3,3), activation = 'leaky_relu', padding = 'same'),
    # Conv2D(64,(3,3), activation = 'leaky_relu', padding = 'same', strides = (2,2)),
    MaxPooling2D((2,2)),
    
    Flatten(),
    BatchNormalization(),
    Dense(32, kernel_regularizer = regularizers.l2(0.015)),
    PReLU(),
    #Dropout(0.5),
    Dense(32, kernel_regularizer = regularizers.l2(0.015)),
    PReLU(),
    # Dropout(0.5),
    Dense(9 , activation = 'softmax')
])

model_3.compile(optimizer= Adam(learning_rate=1e-5), loss = 'binary_crossentropy',metrics = ['accuracy'])
model_3.summary()

2024-04-07 22:43:39.754127: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 22:43:40.037685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 22:43:40.037762: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 22:43:40.039852: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-07 22:43:40.039906: I external/local_xla/xla/stream_executor

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 32, 32, 32)        0

In [4]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation loss
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

tf_callbacks = tf.keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None
)

In [5]:
# Assuming `x_train` and `y_train` are your training data and labels
history = model_3.fit(
    training_set,
    validation_data=test_set,
    #callbacks=[early_stopping,tf_callbacks],
    epochs=100,# Set an arbitrary large number of epochs
    verbose = 1
)

Epoch 1/100


2024-04-07 22:43:57.708094: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-07 22:43:58.698282: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-07 22:44:00.354988: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-07 22:44:03.222709: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f4fd4ce7bd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-07 22:44:03.222764: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-04-07 22:44:03.262878: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712510043.498737  274345 devic

450/450 [==============================] - 85s 161ms/step - loss: 1.9471 - accuracy: 0.2121 - val_loss: 1.8445 - val_accuracy: 0.2456
Epoch 2/100
450/450 [==============================] - 64s 143ms/step - loss: 1.6023 - accuracy: 0.3011 - val_loss: 1.5005 - val_accuracy: 0.3322
Epoch 3/100
450/450 [==============================] - 64s 143ms/step - loss: 1.3880 - accuracy: 0.3919 - val_loss: 1.3211 - val_accuracy: 0.3789
Epoch 4/100
450/450 [==============================] - 76s 168ms/step - loss: 1.2387 - accuracy: 0.4732 - val_loss: 1.1932 - val_accuracy: 0.4461
Epoch 5/100
450/450 [==============================] - 69s 154ms/step - loss: 1.1258 - accuracy: 0.5507 - val_loss: 1.0882 - val_accuracy: 0.5256
Epoch 6/100
450/450 [==============================] - 84s 188ms/step - loss: 1.0295 - accuracy: 0.6178 - val_loss: 1.0101 - val_accuracy: 0.5606
Epoch 7/100
450/450 [==============================] - 76s 169ms/step - loss: 0.9512 - accuracy: 0.6700 - val_loss: 0.9383 - val_accurac

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Assuming `x_test` and `y_test` are your test data and labels
test_loss, test_accuracy = model_3.evaluate(test_set)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


10/10 [==============================] - 1s 61ms/step - loss: 0.7470 - accuracy: 0.8639
Test Loss: 0.7470195293426514, Test Accuracy: 0.8639456033706665


In [ ]:
# Assuming `x_new_data` is your new data
predictions = model_3.predict(test_set)


10/10 [==============================] - 1s 60ms/step


In [ ]:
predictions

array([[2.31487473e-04],
       [1.91054583e-04],
       [4.23968595e-04],
       [1.22222689e-03],
       [1.15974329e-03],
       [5.86320460e-01],
       [3.41303181e-03],
       [6.85614794e-02],
       [9.92758989e-01],
       [4.66471054e-02],
       [1.89056471e-02],
       [2.87959669e-02],
       [6.27887622e-02],
       [7.18021393e-01],
       [5.55972695e-01],
       [3.59264225e-01],
       [8.58780146e-01],
       [4.03961271e-01],
       [2.02992693e-01],
       [1.01047696e-03],
       [8.37257016e-04],
       [3.45150882e-04],
       [2.53277307e-04],
       [3.95951705e-04],
       [8.63489695e-04],
       [1.59829820e-03],
       [1.02435076e-03],
       [1.12151355e-03],
       [2.86851631e-04],
       [3.07253184e-04],
       [4.28573112e-04],
       [2.12384830e-03],
       [1.60761061e-03],
       [8.22175178e-04],
       [3.67835804e-04],
       [2.47826189e-04],
       [1.92125066e-04],
       [3.08654457e-03],
       [1.09239970e-03],
       [5.25143929e-04],


In [ ]:
print(test_set)

In [ ]:
y_test_set = np.concatenate([test_set.next()[1] for i in range(test_set.__len__())])
y_test_set = y_test_set.astype(int)
y_test_set = y_test_set.reshape(len(y_test_set), 1)
print(y_test_set)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [ ]:
predictions_new = (predictions > 0.5)
print(np.concatenate((predictions_new.reshape(len(predictions_new), 1), y_test_set.reshape(len(y_test_set), 1)), 1))

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 1]

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
import numpy as np

# Assuming y_test and y_pred are your true and predicted labels respectively
y_test_decoded = np.argmax(y_test_set, axis=1)
y_pred_decoded = np.argmax(predictions_new, axis=1)

cm = confusion_matrix(y_test_decoded, y_pred_decoded)
acc= accuracy_score(y_test_set, predictions_new)
print(acc,"\n")
print(cm)


[[66 10]
 [10 61]]


0.8639455782312925